# Расчет структуры по нескольким соц.-дем. переменным

## Описание задачи

Посчитаем аудиторию ресурса **Avito** с разбивкой по социально-демографическим переменным:
- Пол
- Возрастные группы

Общие параметры:
- Период: Сентябрь 2021
- География: Россия 100 000+
- Население: 12+
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Список статистик для расчета:
- Reach (reach)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Создаем объекты для работы с API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Формируем задания

Необходимо сформировать следующее задание для API Cross Web:
- расчет аудитории по ресурсу **Avito** с разбивкой по полу и по возрастным группам.

Для переменной **"Пол"** найдем в справочнике название **sex**, для переменной **"Возрастные группы"** - **ageGroup**.

Работа со справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Если указать в задании

```
slices = ["sex", "ageGroup"]
```
то получим каждую из категорий переменной **sex** в разбивке на категории переменной **ageGroup**.

Для нашей задачи необходимо посчитать отдельно разбивку по переменной **sex** и отдельно по **ageGroup**. 

Для этого рассчитаем несколько заданий в цикле.

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-09-01', '2021-09-30')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

### Получим ID ресурса
Для построения отчета необходимо получить идентификатор ресурса __Avito__.

Для этого воспользуемся методом поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Получим идентификатор ресурса **Avito**

In [ ]:
cats.get_resource(resource='Avito')

Таким образом, необходимый идентификатор следующий:

- **Avito**  resourceId = 1028

In [ ]:
# Указываем список срезов, чтобы сформировать структуру расчета
splited_slices = ['sex', 'ageGroup']

# Задаем фильтр по географии, в нашем случае Россия 100 000+
geo_filter = 'cityPop100 = 1'

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр по медиа, в нашем случае это ID ресурса Avito
mart_filter = "crossMediaResourceId = 1028"

# Указываем список статистик для расчета
statistics = ['reach']

# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")
# Для каждого среза формируем задания и отправляем на расчет
for sls in splited_slices:
        
        project_name = f"slice-{sls}"
        slices = [sls]
        # Формируем из указанных параметров задание для API Cross Web в формате JSON для каждого из этапов цикла
        task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter, 
                                     demo_filter, mart_filter, slices, statistics)

        # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
        tsk = {}
        tsk['project_name'] = project_name
        tsk['slice'] = sls
        tsk['task'] = mtask.send_audience_task(task_json)
        tasks.append(tsk)
        time.sleep(2)
        print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])
    df_result['slice'] = t['slice']
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)
df

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('audience-vk'))
df_info = mtask.get_report_info()
df.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()